In [2]:
import os

import sys
sys.path.append("C:\\Users\\User\\PycharmProjects\\GIEBN\\src")
sys.path.append("C:\\Users\\User\\PycharmProjects\\GIEBN")
sys.path.append("C:\\Users\\User\\PycharmProjects\\GIEBN\\venv")

path = os.getcwd()

curr_path = path.replace('\\', '/')
head_place = curr_path.find('GIEBN')
main_path = curr_path[:head_place]+'GIEBN'
main_path

'c:/Users/User/PycharmProjects/GIEBN'

In [86]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import KBinsDiscretizer, FunctionTransformer, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline, make_union
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeClassifier

from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, RFE

from auxiliary.auxiliary import construct_by_kmeans

from pgmpy.factors.discrete.CPD import TabularCPD
from pgmpy.models import BayesianNetwork
import yodo.yodo
import matplotlib.pyplot as plt
from pgmpy.estimators import PC, BicScore, K2Score
from pgmpy.inference import VariableElimination
from bamt.networks.discrete_bn import DiscreteBN
from bamt_inherited.networks.discrete import DiscreteBNGI
import shap
from sklearn import metrics as m
from tqdm import tqdm
from itertools import product
import optuna
from copy import copy

In [30]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [93]:
%run ../common_code.ipynb

In [8]:
data = pd.read_csv("../../datasets/credit.csv")
print(data.columns)
#data = data.reset_index().drop(columns=["index"])
n_features = data.shape[1]-1
n = data.shape[0]
data["default"] = (data["NumberOfTimes90DaysLate"] > 0).astype(int)
data.drop(columns=["NumberOfTimes90DaysLate", 'NumberOfTime60-89DaysPastDueNotWorse', 'SeriousDlqin2yrs',
                   'NumberOfTime30-59DaysPastDueNotWorse'], inplace=True)
n_features = data.shape[1]-1
n = data.shape[0]
data.rename(columns={feat: "".join(feat.split("-")) for feat in data.columns}, inplace=True)
target = "default"
features = data.columns.difference([target]).tolist()

data.head(10)

Index(['RevolvingUtilizationOfUnsecuredLines', 'age',
       'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome',
       'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
       'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
       'NumberOfDependents', 'SeriousDlqin2yrs'],
      dtype='object')


,RevolvingUtilizationOfUnsecuredLines,age,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberRealEstateLoansOrLines,NumberOfDependents,default
0,0.006999,38,0.302150,5440.0,4,1,3,0
1,0.704592,63,0.471441,8000.0,9,1,0,0
2,0.063113,57,0.068586,5000.0,17,0,0,0
3,0.368397,68,0.296273,6250.0,16,2,0,0
4,1.000000,34,0.000000,3500.0,0,0,1,0
5,0.051799,63,0.770687,4700.0,16,1,1,0
6,0.648733,40,0.402060,6600.0,10,2,3,0
7,0.767270,33,0.338390,5800.0,10,0,3,0
8,0.190111,27,0.164605,3960.0,6,0,1,0
9,0.139989,65,0.258397,6400.0,6,1,0,0


In [230]:
data_corr = add_extra_features(data, features, extra_size=0.75, mode='prod')

In [231]:
data_dict_corr = {}

data_tv, data_dict_corr['test'] = train_test_split(data_corr, test_size=0.15, random_state=42,
                                      stratify=data_corr[target])

data_dict_corr['train'], data_dict_corr['val'] = train_test_split(data_tv, test_size=0.15, random_state=42,
                                      stratify=data_tv[target])

features_corr = list(data_corr.columns.difference([target]))

In [232]:
def objective(trial, features, target, data_dict):
    n_estimators= trial.suggest_int("n_estimators", low=params_net['n_estimators'][0], high=params_net['n_estimators'][1], step=1)
    max_depth= trial.suggest_int("max_depth", low=params_net['max_depth'][0], high=params_net['max_depth'][1], step=1)
    learning_rate=trial.suggest_float("learning_rate", low=params_net['learning_rate'][0], high=params_net['learning_rate'][1],log=True)
    
    clf = GradientBoostingClassifier(random_state=42, n_estimators=n_estimators,
                                     max_depth=max_depth, learning_rate=learning_rate)
    clf.fit(data_dict['train'][features], data_dict['train'][target])
    return score_classif(clf, data_dict['val'][features], data_dict['val'][target])

params_net = {
    'n_estimators': [10, 100],
    'max_depth': [3, 10],
    'learning_rate':[1e-5, 1]
}


# Initial model for error calculation

In [233]:
best_base_params = optunize_model(objective, features_corr, target, data_dict_corr)

  0%|          | 0/30 [00:00<?, ?it/s]

In [234]:
best_base_params

{'n_estimators': 58, 'max_depth': 3, 'learning_rate': 0.0810026143211461}

In [235]:
model_corr = GradientBoostingClassifier(random_state=42, **best_base_params)
#model = LogisticRegression()
model_corr.fit(data_dict_corr['train'][features_corr], data_dict_corr['train'][target])

data_errors_corr = measure_error(data_dict_corr['val'], model_corr, features_corr, target, error_col_name='inconf_error', task='classif')

In [236]:
n_features_corr = len(features_corr)
disc_data_corr, pipeline_data_corr = discretize(data_errors_corr, 
                                      features_corr+["inconf_error"], icat=None, icont=list(range(n_features_corr+1)))

categories_corr = extract_categories(disc_data_corr)
encoder = OneHotEncoder(sparse=False)
disc_data_encoded_corr = pd.DataFrame(encoder.fit_transform(X=disc_data_corr), columns=categories_corr, dtype='uint8')
disc_data_encoded_corr.head(1)

,RevolvingUtilizationOfUnsecuredLines_0,RevolvingUtilizationOfUnsecuredLines_1,RevolvingUtilizationOfUnsecuredLines_2,age_0,age_1,age_2,DebtRatio_0,DebtRatio_1,DebtRatio_2,MonthlyIncome_0,...,extra_19_2,extra_20_0,extra_20_1,extra_20_2,extra_21_0,extra_21_1,extra_21_2,inconf_error_0,inconf_error_1,inconf_error_2
0,0,0,1,0,1,0,1,0,0,1,...,0,0,1,0,0,0,1,0,0,1


In [237]:
feat_to_grad_corr = {}
for feat in features_corr+['inconf_error']:
    feat_to_grad_corr[feat]=[grad for grad in disc_data_encoded_corr.columns if feat in grad]


metric_dict = {
    'ROC AUC': lambda x, y: m.roc_auc_score(x,y)*100,
    'log_loss': lambda x, y: m.log_loss(x, y)
}

gbn_fs = DiscreteBNGI('')

gbn_training_pipeline(gbn_fs, disc_data_encoded_corr, 'inconf_error', categories_corr)

2025-04-16 14:01:55,512 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-04-16 14:01:55,513 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-04-16 14:01:55,515 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-04-16 14:01:55,516 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-04-16 14:01:55,518 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-04-16 14:01:55,520 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-04-16 14:01:55,521 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-04-16 14:01:55,524 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-04-16 14:01:55,526 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-04-16 14:01:55,528 | ERROR    | GraphUtil

  0%|          | 0/1000000 [00:00<?, ?it/s]

# Univariate Feature selector

In [238]:
univ_sel = SelectKBest(k=len(features_corr))
univ_sel.fit(data_dict_corr['val'][features_corr], data_dict_corr['val'][target])
df_val_corr_univ = pd.DataFrame({'Feature': univ_sel.feature_names_in_, 'Metric value': univ_sel.pvalues_})

# RFE on decision tree

In [239]:
rfe_sel = RFE(estimator=DecisionTreeClassifier(random_state=42, max_depth=10,), n_features_to_select=1)
#rfe_sel = RFE(estimator=RandomForestClassifier(random_state=42, max_depth=10, n_estimators=50), n_features_to_select=1, )
rfe_sel.fit(data_dict_corr['val'][features_corr], data_dict_corr['val'][target])
df_val_corr_rfe = pd.DataFrame({'Feature': rfe_sel.feature_names_in_, 'Metric value': rfe_sel.ranking_})

# Lasso from Logistic Regression with L1 penalty

In [240]:
lsvc = LogisticRegression(C=0.5, penalty="l1", solver='liblinear')
lsvc.fit(data_dict_corr['val'][features_corr], data_dict_corr['val'][target])
lasso_selector = SelectFromModel(lsvc, prefit=True)

In [241]:
lasso_support = [x for i, x in enumerate(features_corr) if lasso_selector.get_support()[i]]
lasso_support

['MonthlyIncome',
 'NumberOfDependents',
 'NumberOfOpenCreditLinesAndLoans',
 'RevolvingUtilizationOfUnsecuredLines',
 'age',
 'extra_1',
 'extra_10',
 'extra_11',
 'extra_12',
 'extra_13',
 'extra_14',
 'extra_15',
 'extra_17',
 'extra_18',
 'extra_20',
 'extra_21',
 'extra_3',
 'extra_4',
 'extra_5',
 'extra_6',
 'extra_7',
 'extra_8',
 'extra_9']

## GBN selector

In [242]:
df_val_corr_bn=error_based_importance_features_bn(gbn_fs, 'inconf_error', features_corr, feat_to_grad_corr).fillna(0)

100%|██████████| 28/28 [00:09<00:00,  2.92it/s]


## GBN selector run

In [243]:
gbn_res = elimination_experiment(model_corr, data_dict_corr, metric_dict, df_val_corr_bn, direction='asc')

100%|██████████| 28/28 [05:46<00:00, 12.39s/it]


In [244]:
gbn_res.head()

,n_features,last_excluded,ROC AUC,log_loss,"ROC AUC_change, %","log_loss_change, %"
0,28,-,80.36,0.389,0,0
1,27,extra_6,80.365,0.389,0.006,-0.105
2,26,extra_5,80.357,0.389,-0.004,-0.085
3,25,extra_13,80.369,0.389,0.012,-0.115
4,24,extra_12,80.422,0.388,0.077,-0.146


In [246]:
get_best_record(gbn_res, 'ROC AUC')

,n_features,last_excluded,ROC AUC,log_loss,"ROC AUC_change, %","log_loss_change, %"
4,24,extra_12,80.422,0.388,0.077,-0.146


In [247]:
df_corr_excluded_asc_univ=elimination_experiment(model_corr, data_dict_corr, metric_dict,
                       df_val_corr_univ, 'asc')
df_corr_excluded_asc_univ.head()

100%|██████████| 28/28 [05:21<00:00, 11.48s/it]


,n_features,last_excluded,ROC AUC,log_loss,"ROC AUC_change, %","log_loss_change, %"
0,28,-,80.348,0.389,0,0
1,27,NumberOfOpenCreditLinesAndLoans,80.188,0.39,-0.199,0.135
2,26,extra_15,80.15,0.39,-0.247,0.249
3,25,extra_4,80.355,0.389,0.009,-0.067
4,24,NumberRealEstateLoansOrLines,80.356,0.389,0.01,-0.069


In [249]:
get_best_record(df_corr_excluded_asc_univ, 'ROC AUC')

,n_features,last_excluded,ROC AUC,log_loss,"ROC AUC_change, %","log_loss_change, %"
10,18,extra_7,80.549,0.387,0.25,-0.477


In [250]:
df_corr_excluded_rfe=elimination_experiment(model_corr, data_dict_corr, metric_dict,
                       df_val_corr_rfe, 'desc')
df_corr_excluded_rfe.head()

100%|██████████| 28/28 [05:24<00:00, 11.60s/it]


,n_features,last_excluded,ROC AUC,log_loss,"ROC AUC_change, %","log_loss_change, %"
0,28,-,80.349,0.389,0,0
1,27,NumberRealEstateLoansOrLines,80.366,0.389,0.021,-0.109
2,26,NumberOfDependents,80.365,0.389,0.021,-0.107
3,25,extra_14,80.367,0.389,0.022,-0.111
4,24,extra_7,80.369,0.389,0.024,-0.113


In [252]:
get_best_record(df_corr_excluded_rfe, 'ROC AUC')

,n_features,last_excluded,ROC AUC,log_loss,"ROC AUC_change, %","log_loss_change, %"
8,20,extra_13,80.542,0.388,0.24,-0.378


In [253]:
single_run(model_corr, data_dict_corr, lasso_support, target, metric_dict)

[80.316, 0.388]

In [ ]:
#logging
0.5 random, base 79.953, gbn 80.075, univ 79.953, rfe 79.953, lasso: 79.944
0.75 random, base 80.119, gbn 80.55	, univ 80.119, rfe 80.119, lasso: 80.242

0.5 noisy base 80.42, gbn 80.58	, univ 80.42, rfe 80.578, lasso: 80.515
0.75 noisy base 80.264, gbn 80.311, univ 80.264, rfe 80.264, lasso: 80.633

0.5 prod base 80.007, gbn 80.203	, univ 80.022, rfe 80.448, lasso: 80.422
0.75 prod base 80.36, gbn 80.422	, univ 80.549, rfe 80.542, lasso: 80.316
